# Homework
The goal of this homework is to train a simple model for predicting the duration of a ride.

In [1]:
import pandas as pd
import numpy as np
import datetime as dt

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression

In [2]:
df_jan_2023 = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')
df_feb_2023 = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

## Q1. Downloading the data

We'll use the same NYC taxi dataset, but instead of "Green Taxi Trip Records", we'll use "Yellow Taxi Trip Records".

Download the data for January and February 2023.

Read the data for January. How many columns are there?

In [3]:
df_jan_2023

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.30,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.90,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.90,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.10,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.40,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3066761,2,2023-01-31 23:58:34,2023-02-01 00:12:33,NaN,3.05,NaN,None,107,48,0,15.80,0.00,0.5,3.96,0.0,1.0,23.76,NaN,NaN
3066762,2,2023-01-31 23:31:09,2023-01-31 23:50:36,NaN,5.80,NaN,None,112,75,0,22.43,0.00,0.5,2.64,0.0,1.0,29.07,NaN,NaN
3066763,2,2023-01-31 23:01:05,2023-01-31 23:25:36,NaN,4.67,NaN,None,114,239,0,17.61,0.00,0.5,5.32,0.0,1.0,26.93,NaN,NaN
3066764,2,2023-01-31 23:40:00,2023-01-31 23:53:00,NaN,3.15,NaN,None,230,79,0,18.15,0.00,0.5,4.43,0.0,1.0,26.58,NaN,NaN


In [4]:
f'There are: {len(df_jan_2023.columns)} columns'

'There are: 19 columns'

## Q2. Computing duration

Now let's compute the duration variable. It should contain the duration of a ride in minutes.

What's the standard deviation of the trips duration in January?

In [5]:
df_jan_2023['minutes_duration'] = (df_jan_2023['tpep_dropoff_datetime'] - df_jan_2023['tpep_pickup_datetime']).dt.total_seconds().div(60)
df_feb_2023['minutes_duration'] = (df_feb_2023['tpep_dropoff_datetime'] - df_feb_2023['tpep_pickup_datetime']).dt.total_seconds().div(60)

In [6]:
df_jan_2023['minutes_duration']

0           8.433333
1           6.316667
2          12.750000
3           9.616667
4          10.833333
             ...    
3066761    13.983333
3066762    19.450000
3066763    24.516667
3066764    13.000000
3066765    14.400000
Name: minutes_duration, Length: 3066766, dtype: float64

In [7]:
january_duration_std = df_jan_2023['minutes_duration'].std()
f'the standard deviation of the trips duration in January is: {january_duration_std:.2f}'

'the standard deviation of the trips duration in January is: 42.59'

## Q3. Dropping outliers

Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

In [8]:
df_jan_2023['minutes_duration'].describe()

count    3.066766e+06
mean     1.566900e+01
std      4.259435e+01
min     -2.920000e+01
25%      7.116667e+00
50%      1.151667e+01
75%      1.830000e+01
max      1.002918e+04
Name: minutes_duration, dtype: float64

In [9]:
df_jan_filtered = df_jan_2023.loc[(df_jan_2023['minutes_duration'] >= 1) & (df_jan_2023['minutes_duration'] <= 60)]
df_feb_filtered = df_feb_2023.loc[(df_feb_2023['minutes_duration'] >= 1) & (df_feb_2023['minutes_duration'] <= 60)]

In [10]:
df_jan_filtered

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,minutes_duration
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.30,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00,8.433333
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.90,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00,6.316667
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.90,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00,12.750000
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.10,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25,9.616667
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.40,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00,10.833333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3066761,2,2023-01-31 23:58:34,2023-02-01 00:12:33,NaN,3.05,NaN,None,107,48,0,15.80,0.00,0.5,3.96,0.0,1.0,23.76,NaN,NaN,13.983333
3066762,2,2023-01-31 23:31:09,2023-01-31 23:50:36,NaN,5.80,NaN,None,112,75,0,22.43,0.00,0.5,2.64,0.0,1.0,29.07,NaN,NaN,19.450000
3066763,2,2023-01-31 23:01:05,2023-01-31 23:25:36,NaN,4.67,NaN,None,114,239,0,17.61,0.00,0.5,5.32,0.0,1.0,26.93,NaN,NaN,24.516667
3066764,2,2023-01-31 23:40:00,2023-01-31 23:53:00,NaN,3.15,NaN,None,230,79,0,18.15,0.00,0.5,4.43,0.0,1.0,26.58,NaN,NaN,13.000000


In [11]:
percent_left = (df_jan_filtered.shape[0] / df_jan_2023.shape[0]) * 100

In [12]:
f'there are {percent_left:.1f}% of records left after filtering'

'there are 98.1% of records left after filtering'


## Q4. One-hot encoding

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

    Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
    Fit a dictionary vectorizer
    Get a feature matrix from it

What's the dimensionality of this matrix (number of columns)?

In [13]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

df_jan_filtered[categorical] = df_jan_filtered[categorical].astype(str)

/tmp/ipykernel_72454/3345011865.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_jan_filtered[categorical] = df_jan_filtered[categorical].astype(str)


In [14]:
train_dicts = df_jan_filtered[categorical + numerical].to_dict(orient='records')

In [15]:
dict_vectorizer = DictVectorizer()

In [16]:
X_train = dict_vectorizer.fit_transform(train_dicts)

In [17]:
target_feature = "minutes_duration"

In [18]:
y_train = df_jan_filtered[target_feature].values

In [19]:
y_train

array([ 8.43333333,  6.31666667, 12.75      , ..., 24.51666667,
       13.        , 14.4       ])

In [20]:
X_train.shape

(3009173, 516)

## Q5. Training a model

Now let's use the feature matrix from the previous step to train a model.

    Train a plain linear regression model with default parameters, where duration is the response variable
    Calculate the RMSE of the model on the training data

What's the RMSE on train?

In [21]:
lin_model = LinearRegression().fit(X_train, y_train)

In [22]:
lin_model.score(X_train, y_train) # R^2

0.4063140054383355

In [23]:
from sklearn.metrics import root_mean_squared_error

y_true = y_train
y_pred = lin_model.predict(X_train)

RMSE_train = root_mean_squared_error(y_true, y_pred)

In [24]:
f'the RMSE on training set is: {RMSE_train:.2f}'

'the RMSE on training set is: 7.66'

## Q6. Evaluating the model

Now let's apply this model to the validation dataset (February 2023).

What's the RMSE on validation?

In [25]:
df_feb_filtered[categorical] = df_feb_filtered[categorical].astype(str)
valid_dicts = df_feb_filtered[categorical].to_dict(orient='records')

/tmp/ipykernel_72454/2248891913.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_feb_filtered[categorical] = df_feb_filtered[categorical].astype(str)


In [26]:
valid_dicts[:10]

[{'PULocationID': '142', 'DOLocationID': '163'},
 {'PULocationID': '132', 'DOLocationID': '26'},
 {'PULocationID': '161', 'DOLocationID': '145'},
 {'PULocationID': '148', 'DOLocationID': '236'},
 {'PULocationID': '137', 'DOLocationID': '244'},
 {'PULocationID': '263', 'DOLocationID': '141'},
 {'PULocationID': '48', 'DOLocationID': '243'},
 {'PULocationID': '114', 'DOLocationID': '211'},
 {'PULocationID': '114', 'DOLocationID': '249'},
 {'PULocationID': '125', 'DOLocationID': '107'}]

In [27]:
X_valid = dict_vectorizer.transform(valid_dicts)
y_valid = df_feb_filtered[target_feature].values

In [28]:
y_val_pred = lin_model.predict(X_valid)

In [29]:
y_val_pred

array([11.37999149, 50.62565791, 15.03378283, ..., 13.56206798,
       11.15081044, 13.14427429])

In [31]:
RMSE_val = root_mean_squared_error(y_true=y_valid, y_pred=y_val_pred)

In [32]:
f'the RMSE on validation set is: {RMSE_val:.2f}'

'the RMSE on validation set is: 7.82'